In [1]:
# Importe les modules
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 200)

# Projet 8 : Préparation des données

## Sommaire
* [Introduction](#1)
* [Objectif](#2)
* [Renommage des variables](#3)
* [Annuaire de l'éducation](#4)
* [IPS des lycées](#5)
* [Indicateurs de résultat des lycées (IVAL)](#6)

## Introduction<a id='1'></a>

L'ensemble des données provient du Ministère de l'Education Nationale et ont été récupérées sur la plateforme [data.education.gouv.fr](https://data.education.gouv.fr/pages/accueil/).

*Datasets* à notre disposition :
* `fr-en-annuaire-education` ;
* `fr-en-ips-lycees-ap2022` ;
* `fr-en-ips_lycees` ;
* `fr-en-indicateurs-de-resultat-des-lycees-denseignement-general-et-technologique` ;
* `fr-en-indicateurs-de-resultat-des-lycees-denseignement-professionnels`.

Ils ont été mis à jour en 2023 et ils contiennent des données jusqu'en 2022.

Un fichier supplémentaire, permettant à Power BI de créer des visuels de type « Carte de formes », a été généré à partir de [FranceGEOJSON](https://france-geojson.gregoiredavid.fr/).
> Le projet Github france-geojson propose au format GeoJSON les cartes des régions, départements, arrondissements, cantons et communes de France (métropole et départements d'outre-mer) à partir des données publiées par l'IGN et l'INSEE.

Le fichier `geojson` a ensuite été converti au format `json` grâce au site [Mapshaper](https://mapshaper.org/) pour être utilisable par Power BI. Il se nomme : `fr-departements-shape-map`.

## Objectif<a id='2'></a>

Dans ce *notebook* nous allons nettoyer et préparer les données nécessaire à la création du rapport Power BI.

Principales étapes :
* Renommage et tri des variables.
* Suppression des DROM-COM : l'analyse ne portera que sur la France métropolitaine.
* Vérification des valeurs manquantes, des doublons et des types.
* Concaténation des deux fichiers des IPS.
* Suppression des années inutiles dans les fichiers des indicateurs de résultat (les IPS ne sont disponibles que depuis la rentrée 2016-2017).
* Autre corrections diverses.
* Exportation des fichiers.

## Renommage des variables<a id='3'></a>

Les *datasets* comprennent de nombreuses variables avec des accents, des apostrophes, etc.. Pour faciliter la lecture et les manipulations, on commence par créer une fonction qui convertit les variables selon la convention *snake case*.

In [2]:
def snake_case (df):
    """Renomme les variables d'un DataFrame selon la convention snake case."""
    df.columns = (
        df.columns
        .str.lower()
        .str.replace('\'', '_')
        .str.replace(' - ', '_')
        .str.replace('-', '_')
        .str.replace(' ', '_')
        .str.normalize('NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
    )

## Annuaire de l'éducation<a id='4'></a>

A partir de l'annuaire, nous allons créer un jeu de données contenant toutes les informations géographiques des établissements. Ainsi, ces informations pourront être supprimées des autres jeux de données.

### Importation des données et aperçu

In [3]:
df_annuaire = pd.read_csv(
    './datasets/raw/fr-en-annuaire-education.csv', sep=';', low_memory=False)

snake_case(df_annuaire)

df_annuaire.head()

,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,statut_public_prive,adresse_1,adresse_2,adresse_3,code_postal,code_commune,nom_commune,code_departement,code_academie,code_region,ecole_maternelle,ecole_elementaire,voie_generale,voie_technologique,voie_professionnelle,telephone,fax,web,mail,restauration,hebergement,ulis,apprentissage,segpa,section_arts,section_cinema,section_theatre,section_sport,section_internationale,section_europeenne,lycee_agricole,lycee_militaire,lycee_des_metiers,post_bac,appartenance_education_prioritaire,greta,siren_siret,nombre_d_eleves,fiche_onisep,position,type_contrat_prive,libelle_departement,libelle_academie,libelle_region,coordonnee_x,coordonnee_y,epsg,nom_circonscription,latitude,longitude,precision_localisation,date_ouverture,date_maj_ligne,etat,ministere_tutelle,etablissement_multi_lignes,rpi_concentre,rpi_disperse,code_nature,libelle_nature,code_type_contrat_prive,pial,etablissement_mere,type_rattachement_etablissement_mere,code_bassin_formation,libelle_bassin_formation
0,0641141T,ECOLE PRIMAIRE PUBLIQUE HERAURITZ,Ecole,Public,Quartier Herauritz,NaN,64480 USTARITZ,64480,64547,Ustaritz,064,4,75,1.0,1.0,NaN,NaN,NaN,0559930682,NaN,NaN,ce.0641141T@ac-bordeaux.fr,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21640547200038,99.0,NaN,"43.41939759772092, -1.4672331951877942",SANS OBJET,Pyrénées-Atlantiques,Bordeaux,Nouvelle-Aquitaine,338242.9,6268045.1,EPSG:2154,Circonscription d'inspection du 1er degré de B...,43.419398,-1.467233,PLAQUE_ADRESSE,1970-02-04,2023-06-13,OUVERT,MINISTERE DE L'EDUCATION NATIONALE,0,0.0,NaN,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0641392R,NaN,NaN,04503,PAYS BASQUE
1,0641147Z,ECOLE PRIMAIRE DU BOURG,Ecole,Public,BOURG,NaN,64130 VIODOS ABENSE DE BAS,64130,64559,Viodos-Abense-de-Bas,064,4,75,1.0,1.0,NaN,NaN,NaN,0559191405,NaN,NaN,ce.0641147Z@ac-bordeaux.fr,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21640559700024,38.0,NaN,"43.24304294866094, -0.8803405181620836",SANS OBJET,Pyrénées-Atlantiques,Bordeaux,Nouvelle-Aquitaine,384765.4,6245952.5,EPSG:2154,Circonscription d'inspection du 1er degré de S...,43.243043,-0.880341,PLAQUE_ADRESSE,1970-02-06,2023-06-13,OUVERT,MINISTERE DE L'EDUCATION NATIONALE,0,0.0,NaN,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0640039V,NaN,NaN,NaN,NaN
2,0641170Z,ECOLE ELEMENTAIRE PUBLIQUE,Ecole,Public,Rue de l'Ardoisière,NaN,64490 BEDOUS,64490,64104,Bedous,064,4,75,0.0,1.0,NaN,NaN,NaN,0559347476,NaN,NaN,ce.0641170Z@ac-bordeaux.fr,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21640104200033,57.0,NaN,"42.99894867732615, -0.5992616889748449",SANS OBJET,Pyrénées-Atlantiques,Bordeaux,Nouvelle-Aquitaine,406346.9,6217760.6,EPSG:2154,Circonscription d'inspection du 1er degré d'Ol...,42.998949,-0.599262,PLAQUE_ADRESSE,1970-06-03,2023-06-13,OUVERT,MINISTERE DE L'EDUCATION NATIONALE,0,0.0,064411,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0640016V,NaN,NaN,04502,CENTRE
3,0641213W,ECOLE PRIMAIRE LE BRAOU,Ecole,Public,4 rue de la Barthe,NaN,64200 BIARRITZ,64200,64122,Biarritz,064,4,75,1.0,1.0,NaN,NaN,NaN,0559239698,NaN,NaN,ce.0641213W@ac-bordeaux.fr,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21640122400052,121.0,NaN,"43.474543516529586, -1.5420084781313812",SANS OBJET,Pyrénées-Atlantiques,Bordeaux,Nouvelle-Aquitaine,332546.6,6274510.0,EPSG:2154,Circonscription d'inspection du 1er degré de B...,43.474544,-1.542008,PLAQUE_ADRESSE,1970-06-08,2023-06-13,OUVERT,MINISTERE DE L'EDUCATION NATIONALE,0,0.0,NaN,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0641414P,NaN,NaN,04503,PAYS BASQUE
4,0641217A,ECOLE ELEMENTAIRE PUBLIQUE EVARISTE GALOIS,Ecole,Public,10 AVENUE DE MAIGNON,NaN,64600 ANGLET,64600,64024,Anglet,064,4,75,0.0,1.0,NaN,NaN,NaN,0559631117,NaN,NaN,ce.0641217A@ac-bordeaux.fr,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21640024200139,202.0,NaN,"43.47960001507585, -1.5069504671877214",SANS OBJET,Pyrénées-Atlantiques,Bordeaux,Nouvelle-Aquitaine,335411.8,6274908.6,EPSG:

### Dimensions

In [4]:
print(f'Dimensions du DataFrame : {df_annuaire.shape}')

Dimensions du DataFrame : (68962, 69)


### Tri des variables

Il est compliqué de travailler sur un nombre aussi important de variables. On commence donc par supprimer celles dont nous sommes sûre de ne pas avoir besoin.

In [5]:
df_annuaire.columns

Index(['identifiant_de_l_etablissement', 'nom_etablissement',
       'type_etablissement', 'statut_public_prive', 'adresse_1', 'adresse_2',
       'adresse_3', 'code_postal', 'code_commune', 'nom_commune',
       'code_departement', 'code_academie', 'code_region', 'ecole_maternelle',
       'ecole_elementaire', 'voie_generale', 'voie_technologique',
       'voie_professionnelle', 'telephone', 'fax', 'web', 'mail',
       'restauration', 'hebergement', 'ulis', 'apprentissage', 'segpa',
       'section_arts', 'section_cinema', 'section_theatre', 'section_sport',
       'section_internationale', 'section_europeenne', 'lycee_agricole',
       'lycee_militaire', 'lycee_des_metiers', 'post_bac',
       'appartenance_education_prioritaire', 'greta', 'siren_siret',
       'nombre_d_eleves', 'fiche_onisep', 'position', 'type_contrat_prive',
       'libelle_departement', 'libelle_academie', 'libelle_region',
       'coordonnee_x', 'coordonnee_y', 'epsg', 'nom_circonscription',
       'latitude',

In [6]:
df_annuaire = df_annuaire.drop(
    columns=[
        'statut_public_prive', 'code_commune', 'ecole_maternelle',
        'ecole_elementaire', 'voie_generale', 'voie_technologique',
        'voie_professionnelle', 'telephone', 'fax', 'web', 'mail',
        'restauration', 'hebergement', 'ulis', 'apprentissage', 'segpa',
        'section_arts', 'section_cinema', 'section_theatre', 'section_sport',
        'section_internationale', 'section_europeenne', 'lycee_agricole',
        'lycee_militaire', 'lycee_des_metiers', 'post_bac',
        'appartenance_education_prioritaire', 'greta', 'siren_siret', 'fiche_onisep',
        'position', 'type_contrat_prive', 'coordonnee_x', 'coordonnee_y', 'epsg',
        'nom_circonscription', 'precision_localisation', 'date_ouverture',
        'date_maj_ligne', 'etat', 'ministere_tutelle', 'rpi_concentre', 'rpi_disperse',
        'code_nature', 'libelle_nature', 'code_type_contrat_prive', 'pial',
        'etablissement_mere', 'type_rattachement_etablissement_mere',
        'code_bassin_formation', 'libelle_bassin_formation'
    ]
)

In [7]:
df_annuaire.head()

,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,adresse_1,adresse_2,adresse_3,code_postal,nom_commune,code_departement,code_academie,code_region,nombre_d_eleves,libelle_departement,libelle_academie,libelle_region,latitude,longitude,etablissement_multi_lignes
0,0641141T,ECOLE PRIMAIRE PUBLIQUE HERAURITZ,Ecole,Quartier Herauritz,NaN,64480 USTARITZ,64480,Ustaritz,064,4,75,99.0,Pyrénées-Atlantiques,Bordeaux,Nouvelle-Aquitaine,43.419398,-1.467233,0
1,0641147Z,ECOLE PRIMAIRE DU BOURG,Ecole,BOURG,NaN,64130 VIODOS ABENSE DE BAS,64130,Viodos-Abense-de-Bas,064,4,75,38.0,Pyrénées-Atlantiques,Bordeaux,Nouvelle-Aquitaine,43.243043,-0.880341,0
2,0641170Z,ECOLE ELEMENTAIRE PUBLIQUE,Ecole,Rue de l'Ardoisière,NaN,64490 BEDOUS,64490,Bedous,064,4,75,57.0,Pyrénées-Atlantiques,Bordeaux,Nouvelle-Aquitaine,42.998949,-0.599262,0
3,0641213W,ECOLE PRIMAIRE LE BRAOU,Ecole,4 rue de la Barthe,NaN,64200 BIARRITZ,64200,Biarritz,064,4,75,121.0,Pyrénées-Atlantiques,Bordeaux,Nouvelle-Aquitaine,43.474544,-1.542008,0
4,0641217A,ECOLE ELEMENTAIRE PUBLIQUE EVARISTE GALOIS,Ecole,10 AVENUE DE MAIGNON,NaN,64600 ANGLET,64600,Anglet,064,4,75,202.0,Pyrénées-Atlantiques,Bordeaux,Nouvelle-Aquitaine,43.479600,-1.506950,0


C'est plus clair. On renomme l'identifiant de l'établissement en « uai ».

In [8]:
df_annuaire = df_annuaire.rename(columns={'identifiant_de_l_etablissement': 'uai'})

Voyons à quoi correspond le type d'établissement.

In [9]:
df_annuaire['type_etablissement'].unique()

array(['Ecole', 'Collège', 'Lycée', 'EREA', 'Autre', 'Médico-social',
       'Information et orientation', 'Service Administratif', nan],
      dtype=object)

On supprime les enregistrements autres que « Lycée » et on conserve les valeurs manquantes. Dans un deuxième temps, on supprime la variable « type_etablissement ».

In [10]:
# Liste les index des enregistrements à supprimer
index_names = df_annuaire.loc[
    (df_annuaire['type_etablissement'] != 'Lycée')
    & (~df_annuaire['type_etablissement'].isna())
].index

# Supprime les enregistrements
df_annuaire = df_annuaire.drop(index=index_names).reset_index(drop=True)

# Supprime "type_etablissement"
df_annuaire = df_annuaire.drop(columns=['type_etablissement'])

del index_names

On va étudier une deuxième variable pour comprendre à quoi elle correspond. Il s'agit de « etablissement_multi_lignes ».

In [11]:
df_annuaire['etablissement_multi_lignes'].value_counts()

0    5895
1     136
Name: etablissement_multi_lignes, dtype: int64

136 établissements ont la valeur « 1 ». On recherche un UAI concerné.

In [12]:
df_annuaire.loc[df_annuaire['etablissement_multi_lignes'] == 1].head()

,uai,nom_etablissement,adresse_1,adresse_2,adresse_3,code_postal,nom_commune,code_departement,code_academie,code_region,nombre_d_eleves,libelle_departement,libelle_academie,libelle_region,latitude,longitude,etablissement_multi_lignes
14,0220107X,Lycée Notre-Dame de Campostal - site de Rostrenen,Place du Bourg Coz,NaN,NaN,22110,Rostrenen,022,14,53,168.0,Côtes-d'Armor,Rennes,Bretagne,48.235347,-3.313970,1
42,0572340J,Lycée professionnel Antoine Gapp - Site de For...,2 avenue du Général Passaga,NaN,NaN,57600,Forbach,057,12,44,237.0,Moselle,Nancy-Metz,Grand Est,49.184913,6.901370,1
67,0570279U,Lycée professionnel La Providence,1 rue des Frères Mesguin,NaN,NaN,57260,Dieuze,057,12,44,405.0,Moselle,Nancy-Metz,Grand Est,48.811707,6.720350,1
79,0772153X,Lycée technologique Jean Rose - secteur tertia...,20 rue de Châage,NaN,NaN,77109,Meaux,077,24,11,277.0,Seine-et-Marne,Créteil,Ile-de-France,48.963587,2.879744,1
81,0772153X,Lycée technologique Jean Rose - secteur hôtell...,12 rue de la Visitation,NaN,NaN,77100,Meaux,077,24,11,277.0,Seine-et-Marne,Créteil,Ile-de-France,48.963587,2.879744,1


In [13]:
df_annuaire.loc[df_annuaire['uai'] == '0570279U']

,uai,nom_etablissement,adresse_1,adresse_2,adresse_3,code_postal,nom_commune,code_departement,code_academie,code_region,nombre_d_eleves,libelle_departement,libelle_academie,libelle_region,latitude,longitude,etablissement_multi_lignes
67,0570279U,Lycée professionnel La Providence,1 rue des Frères Mesguin,NaN,NaN,57260,Dieuze,057,12,44,405.0,Moselle,Nancy-Metz,Grand Est,48.811707,6.72035,1
2223,0570279U,Annexe du lycée professionnel La Providence de...,3 rue Goussaud,NaN,NaN,57000,Metz,057,12,44,405.0,Moselle,Nancy-Metz,Grand Est,48.811707,6.72035,1
5590,0570279U,Annexe du lycée professionnel La Providence de...,10 rue de l'Abbé Heydel,NaN,NaN,57800,Freyming-Merlebach,057,12,44,405.0,Moselle,Nancy-Metz,Grand Est,48.811707,6.72035,1


Il s'agit de lycées multi-sites ou avec des annexes. Il faudra par contre être vigilant dans les analyses car le nombre d'élèves est à chaque fois répété. On conserve cette variable multi-site pour pouvoir les filtrer si besoin.

Pour finir cette partie sur le tri, nous supprimons les enregistrements des DROM-COM.

In [14]:
df_annuaire['code_departement'].unique()

array(['074', '038', '073', '007', '026', '037', '063', '076', '069',
       '075', '056', '044', '014', '022', '091', '077', '054', '083',
       '085', '032', '072', '059', '095', '973', '972', '084', '974',
       '012', '046', '004', '057', '013', '092', '031', '051', '094',
       '049', '02B', '029', '011', '080', '976', '003', '042', '971',
       '087', '062', '041', '005', '045', '035', '009', '034', '036',
       '025', '030', '006', '015', '088', '090', '068', '067', '061',
       '001', '078', '002', '093', '065', '048', '027', '053', '071',
       '082', '060', '052', '021', '070', '017', '033', '028', '055',
       '050', '988', '047', '064', '008', '010', '018', '024', '039',
       '043', '058', '066', '081', '086', '978', '019', '987', '986',
       '016', '089', '023', '040', '02A', '079', '977', '975'],
      dtype=object)

In [15]:
# Liste les index des enregistrements à supprimer
index_names = df_annuaire.loc[
    df_annuaire['code_departement'].str.startswith('9')].index

# Supprime les enregistrements
df_annuaire = df_annuaire.drop(index=index_names).reset_index(drop=True)

del index_names

### Valeurs manquantes

In [16]:
df_annuaire.isna().sum()

uai                              0
nom_etablissement                0
adresse_1                        5
adresse_2                     3542
adresse_3                     4467
code_postal                      0
nom_commune                      0
code_departement                 0
code_academie                    0
code_region                      0
nombre_d_eleves               2203
libelle_departement              0
libelle_academie                 0
libelle_region                   0
latitude                         5
longitude                        5
etablissement_multi_lignes       0
dtype: int64

Il n'y a pas de valeurs manquantes dans les UAI et c'est le principal. Il manque un nombre important d'information sur les effectifs, mais nous ne pouvons pas supprimer les lignes.

### Doublons

In [17]:
df_annuaire.duplicated().sum()

2

Deux lignes sont dupliquées, on les supprime.

In [18]:
df_annuaire = df_annuaire.drop_duplicates().reset_index(drop=True)

### Type des données

In [19]:
df_annuaire.dtypes

uai                            object
nom_etablissement              object
adresse_1                      object
adresse_2                      object
adresse_3                      object
code_postal                     int64
nom_commune                    object
code_departement               object
code_academie                   int64
code_region                     int64
nombre_d_eleves               float64
libelle_departement            object
libelle_academie               object
libelle_region                 object
latitude                      float64
longitude                     float64
etablissement_multi_lignes      int64
dtype: object

On convertit quelques variables pour optimiser l'utilisation de la mémoire même si on ne va pas effectuer d'analyses avec Pandas.<br>
Python considère les valeurs nulles comme des nombres décimaux, on ne peut pas convertir le nombre d'élèves en entier en utilisant simplement le type `int16`, on utilise donc `pd.Int16Dtype()` à la place.

In [20]:
df_annuaire = df_annuaire.astype(
    {
        'code_postal': 'object',
        'code_departement': 'category',
        'code_academie': 'category',
        'code_region': 'category',
        'etablissement_multi_lignes': 'boolean',
        'nombre_d_eleves': pd.Int16Dtype()
    }
)

### Exportation CSV

In [21]:
df_annuaire.to_csv('./datasets/position-geographique-lycees.csv',
                    index=False, decimal=',', quoting=1)

## IPS des lycées<a id='5'></a>

### Importation des données et aperçu

In [22]:
df_ips_2022 = pd.read_csv('./datasets/raw/fr-en-ips-lycees-ap-2022.csv', sep=';')

snake_case(df_ips_2022)

df_ips_2022.head()

,rentree_scolaire,academie,code_du_departement,departement,uai,nom_de_l_etablissment,code_insee_de_la_commune,nom_de_la_commune,secteur,type_de_lycee,effectifs_voie_gt,effectifs_voie_pro,effectifs_ensemble_gt_pro,ips_voie_gt,ips_voie_pro,ips_ensemble_gt_pro,ecart_type_de_l_ips_voie_gt,ecart_type_de_l_ips_voie_pro
0,2022-2023,LYON,001,AIN,0010001W,LYCEE PROFESSIONNEL ALEXANDRE BERARD LYCEE DES...,01004,AMBERIEU EN BUGEY,public,LP,NaN,583.0,583.0,NaN,91.5,91.5,NaN,27.2
1,2022-2023,LYON,001,AIN,0010006B,LYCEE POLYVALENT SAINT EXUPERY LYCEE DES METIE...,01033,VALSERHONE,public,LPO,556.0,334.0,890.0,99.2,84.2,93.6,33.3,24.8
2,2022-2023,LYON,001,AIN,0010013J,LYCEE GENERAL LALANDE,01053,BOURG EN BRESSE,public,LEGT,1028.0,NaN,1028.0,123.0,NaN,123.0,34.7,NaN
3,2022-2023,LYON,001,AIN,0010016M,LYCEE POLYVALENT JOSEPH MARIE CARRIAT,01053,BOURG EN BRESSE,public,LPO,1362.0,335.0,1697.0,111.7,98.8,109.2,32.9,30.2
4,2022-2023,LYON,001,AIN,0010069V,LYCEE GENERAL ET TECHNOLOGIQUE PRIVE ENSEMBLE ...,01034,BELLEY,privé sous contrat,LEGT,250.0,NaN,250.0,115.8,NaN,115.8,27.5,NaN


In [23]:
df_ips_2017_2021 = pd.read_csv('./datasets/raw/fr-en-ips-lycees-2017-2021.csv', sep=';')

snake_case(df_ips_2017_2021)

df_ips_2017_2021.head()

,rentree_scolaire,academie,code_du_departement,departement,uai,nom_de_l_etablissment,code_insee_de_la_commune,nom_de_la_commune,secteur,type_de_lycee,ips_voie_gt,ips_voie_pro,ips_ensemble_gt_pro,ecart_type_de_l_ips_voie_gt,ecart_type_de_l_ips_voie_pro
0,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0771995A,LYCEE PROFESSIONNEL LE CHAMP DE CLAYE LYCEE DE...,77118.0,CLAYE SOUILLY,public,LP,NaN,87.4,87.4,NaN,NaN
1,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0772127U,LYCEE GENERAL ET TECHNOLOGIQUE GALILEE,77122.0,COMBS LA VILLE,public,LEGT,115.6,NaN,115.6,NaN,NaN
2,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0772151V,LYCEE PROFESSIONNEL PRIVE LES SINOPLIES,77390.0,ROISSY EN BRIE,privé sous contrat,LP,NaN,114.2,114.2,NaN,NaN
3,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0772188K,LYCEE GENERAL ET TECHNOLOGIQUE PIERRE MENDES F...,77445.0,SAVIGNY LE TEMPLE,public,LEGT,99.4,NaN,99.4,NaN,NaN
4,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0772292Y,LYCEE POLYVALENT MARTIN LUTHER KING,77058.0,BUSSY SAINT GEORGES,public,LEGT,121.3,NaN,121.3,NaN,NaN


### Dimensions

In [24]:
print(
    f'Dimensions des DataFrames :\n'
    f'- df_ips_2022 : {df_ips_2022.shape}\n'
    f'- df_ips_2017_2021 : {df_ips_2017_2021.shape}'
)

Dimensions des DataFrames :
- df_ips_2022 : (3598, 18)
- df_ips_2017_2021 : (21777, 15)


Le fichier de 2022 a trois variables de plus que l'ancien fichier.

### Tri des variables

On les identifie dans le fichier de 2022.

In [25]:
variables_differences = set(df_ips_2022.columns) - set(df_ips_2017_2021.columns)
variables_differences

{'effectifs_ensemble_gt_pro', 'effectifs_voie_gt', 'effectifs_voie_pro'}

Il s'agit des effectifs détaillés par voie, on les supprime pour pouvoir concaténer les *DataFrames*.

In [26]:
df_ips_2022 = df_ips_2022.drop(columns=variables_differences)
del variables_differences

On concatène les données.

In [27]:
df_ips_2022.columns

Index(['rentree_scolaire', 'academie', 'code_du_departement', 'departement',
       'uai', 'nom_de_l_etablissment', 'code_insee_de_la_commune',
       'nom_de_la_commune', 'secteur', 'type_de_lycee', 'ips_voie_gt',
       'ips_voie_pro', 'ips_ensemble_gt_pro', 'ecart_type_de_l_ips_voie_gt',
       'ecart_type_de_l_ips_voie_pro'],
      dtype='object')

In [28]:
df_ips_2017_2021.columns

Index(['rentree_scolaire', 'academie', 'code_du_departement', 'departement',
       'uai', 'nom_de_l_etablissment', 'code_insee_de_la_commune',
       'nom_de_la_commune', 'secteur', 'type_de_lycee', 'ips_voie_gt',
       'ips_voie_pro', 'ips_ensemble_gt_pro', 'ecart_type_de_l_ips_voie_gt',
       'ecart_type_de_l_ips_voie_pro'],
      dtype='object')

In [29]:
df_ips = pd.concat([df_ips_2017_2021, df_ips_2022]).reset_index(drop=True)
del df_ips_2022, df_ips_2017_2021

df_ips.head()

,rentree_scolaire,academie,code_du_departement,departement,uai,nom_de_l_etablissment,code_insee_de_la_commune,nom_de_la_commune,secteur,type_de_lycee,ips_voie_gt,ips_voie_pro,ips_ensemble_gt_pro,ecart_type_de_l_ips_voie_gt,ecart_type_de_l_ips_voie_pro
0,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0771995A,LYCEE PROFESSIONNEL LE CHAMP DE CLAYE LYCEE DE...,77118.0,CLAYE SOUILLY,public,LP,NaN,87.4,87.4,NaN,NaN
1,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0772127U,LYCEE GENERAL ET TECHNOLOGIQUE GALILEE,77122.0,COMBS LA VILLE,public,LEGT,115.6,NaN,115.6,NaN,NaN
2,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0772151V,LYCEE PROFESSIONNEL PRIVE LES SINOPLIES,77390.0,ROISSY EN BRIE,privé sous contrat,LP,NaN,114.2,114.2,NaN,NaN
3,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0772188K,LYCEE GENERAL ET TECHNOLOGIQUE PIERRE MENDES F...,77445.0,SAVIGNY LE TEMPLE,public,LEGT,99.4,NaN,99.4,NaN,NaN
4,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0772292Y,LYCEE POLYVALENT MARTIN LUTHER KING,77058.0,BUSSY SAINT GEORGES,public,LEGT,121.3,NaN,121.3,NaN,NaN


Nous n'avons pas besoin des variables relatives aux informations géographiques car ces dernières sont présentes dans le fichier CSV généré précédemment.<br>
Mais avant de les enlever, on supprime les enregistrements des DROM-COM.

In [30]:
df_ips['code_du_departement'].unique()

array([77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0,
       88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, nan, 971.0, 972.0,
       973.0, 974.0, 976.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0,
       10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 21.0,
       22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0,
       33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0,
       44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0,
       55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0,
       66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0,
       '001', '002', '003', '004', '005', '006', '007', '008', '009',
       '010', '011', '012', '013', '014', '015', '016', '017', '018',
       '019', '021', '022', '023', '024', '025', '026', '027', '028',
       '029', '030', '031', '032', '033', '034', '035', '036', '037',
       '038', '039', '040', '041', '042', '043', '044', '045',

In [31]:
# Crée la liste des départements à supprimer
index_names = df_ips.loc[
    ((df_ips['code_du_departement'].str.startswith('9')) 
     & (df_ips['code_du_departement'].str.len() == 3))
     | ((df_ips['code_du_departement'].str.startswith('9'))
        & (df_ips['code_du_departement'].str.len() == 5))
].index

# Supprime les enregistrements
df_ips = df_ips.drop(index=index_names).reset_index(drop=True)
del index_names

In [32]:
df_ips.columns

Index(['rentree_scolaire', 'academie', 'code_du_departement', 'departement',
       'uai', 'nom_de_l_etablissment', 'code_insee_de_la_commune',
       'nom_de_la_commune', 'secteur', 'type_de_lycee', 'ips_voie_gt',
       'ips_voie_pro', 'ips_ensemble_gt_pro', 'ecart_type_de_l_ips_voie_gt',
       'ecart_type_de_l_ips_voie_pro'],
      dtype='object')

In [33]:
df_ips = df_ips.drop(
    columns={'academie', 'code_du_departement', 'departement',
             'nom_de_l_etablissment', 'code_insee_de_la_commune',
             'nom_de_la_commune', 'ips_ensemble_gt_pro'}
)

df_ips.head()

,rentree_scolaire,uai,secteur,type_de_lycee,ips_voie_gt,ips_voie_pro,ecart_type_de_l_ips_voie_gt,ecart_type_de_l_ips_voie_pro
0,2018-2019,0771995A,public,LP,NaN,87.4,NaN,NaN
1,2018-2019,0772127U,public,LEGT,115.6,NaN,NaN,NaN
2,2018-2019,0772151V,privé sous contrat,LP,NaN,114.2,NaN,NaN
3,2018-2019,0772188K,public,LEGT,99.4,NaN,NaN,NaN
4,2018-2019,0772292Y,public,LEGT,121.3,NaN,NaN,NaN


La rentrée scolaire est une plage de deux années. Nous allons donc « couper » en deux cette variable en gardant d'un côté la rentrée et de l'autre l'année car c'est cette information que l'on retrouve dans le fichier des indicateurs de résultat.

In [34]:
df_ips[['rentree', 'annee']] = (
    df_ips['rentree_scolaire'].str.split('-', expand=True)
)
df_ips = df_ips.drop(columns='rentree_scolaire')

On retouche les valeurs présentes dans « secteur » et « type_de_lycee » en vérifiant au préalable les valeurs qu'elles contiennent.

In [35]:
df_ips[['secteur', 'type_de_lycee']].value_counts()

secteur             type_de_lycee
public              LEGT             6128
                    LP               5564
                    LPO              5058
privé sous contrat  LEGT             4576
                    LP               2514
                    LPO              1396
dtype: int64

In [36]:
df_ips['secteur'] = df_ips['secteur'].map(
    {'public': 'Public', 'privé sous contrat': 'Privé'}
)
df_ips['type_de_lycee'] = df_ips['type_de_lycee'].map(
    {'LEGT': 'Lycée général et technologique', 'LP': 'Lycée professionnel',
     'LPO': 'Lycée polyvalent'}
)

On renomme les écarts-types et on modifie l'ordre des colonnes.

In [37]:
df_ips = df_ips.rename(
    columns={'ecart_type_de_l_ips_voie_gt': 'ecart_type_ips_voie_gt',
             'ecart_type_de_l_ips_voie_pro': 'ecart_type_ips_voie_pro'}
)

df_ips = df_ips[
    ['annee', 'rentree', 'uai', 'secteur', 'type_de_lycee', 'ips_voie_gt',
     'ips_voie_pro', 'ecart_type_ips_voie_gt', 'ecart_type_ips_voie_pro']
]

### Valeurs manquantes

In [38]:
df_ips.isna().sum()

annee                          0
rentree                        0
uai                            0
secteur                        0
type_de_lycee                  0
ips_voie_gt                 8078
ips_voie_pro               10704
ecart_type_ips_voie_gt     15459
ecart_type_ips_voie_pro    16986
dtype: int64

Il a des valeurs manquantes dans les IPS mais cela n'est pas un problème. En effet, il est normal qu'un lycée professionnel n'ait pas de valeur d'IPS pour la voie générale. On peut vérifier si pour certains établissements il manque les IPS dans toutes les voies.

In [39]:
len(df_ips.loc[(df_ips['ips_voie_gt'].isna()) & (df_ips['ips_voie_pro'].isna())])

0

Aucun lycée n'est concerné. En ce qui concerne les écarts-types, on en reste là car ils n'étaient pas calculés par le passé.

### Doublons

In [40]:
df_ips.duplicated().sum()

0

Il n'y a pas de doublons.

### Type des données

In [41]:
df_ips.dtypes

annee                       object
rentree                     object
uai                         object
secteur                     object
type_de_lycee               object
ips_voie_gt                float64
ips_voie_pro               float64
ecart_type_ips_voie_gt     float64
ecart_type_ips_voie_pro    float64
dtype: object

On convertit une partie des variables en type catégorie.

In [42]:
df_ips = df_ips.astype(
    {'annee': 'category', 'rentree': 'category', 'secteur': 'category',
     'type_de_lycee': 'category'}
)

### Exportation CSV

In [43]:
df_ips.to_csv('./datasets/ips-lycees-2017-2022-hors-drom-com.csv',
                    index=False, decimal=',', quoting=1)

## Indicateurs de résultat des lycées (IVAL)<a id='6'></a>

### Importation des données et aperçu

In [44]:
df_ival_gt = pd.read_csv(
    './datasets/raw/fr-en-indicateurs-resultat-lycees-generaux-technologiques-2012-2022.csv',
    sep=';', low_memory=False)

snake_case(df_ival_gt)

df_ival_gt.head()

,etablissement,annee,ville,uai,code_commune,academie,departement,secteur,presents_l,presents_es,presents_s,presents_stg,presents_sti2d,presents_std2a,presents_stmg,presents_sti,presents_stl,presents_st2s,presents_tmd,presents_sthr,presents_toutes_series,taux_de_reussite_l,taux_de_reussite_es,taux_de_reussite_s,taux_de_reussite_stg,taux_de_reussite_sti2d,taux_de_reussite_std2a,taux_de_reussite_stmg,taux_de_reussite_sti,taux_de_reussite_stl,taux_de_reussite_st2s,taux_de_reussite_tmd,taux_de_reussite_sthr,taux_de_reussite_toutes_series,taux_de_reussite_attendu_acad_l,taux_de_reussite_attendu_acad_es,taux_de_reussite_attendu_acad_s,taux_de_reussite_attendu_acad_stg,taux_de_reussite_attendu_acad_sti2d,taux_de_reussite_attendu_acad_std2a,taux_de_reussite_attendu_acad_stmg,taux_de_reussite_attendu_acad_sti,taux_de_reussite_attendu_acad_stl,taux_de_reussite_attendu_acad_st2s,taux_de_reussite_attendu_acad_tmd,taux_de_reussite_attendu_acad_sthr,taux_de_reussite_attendu_acad_toutes_series,taux_de_reussite_attendu_france_l,taux_de_reussite_attendu_france_es,taux_de_reussite_attendu_france_s,taux_de_reussite_attendu_france_stg,taux_de_reussite_attendu_france_sti2d,taux_de_reussite_attendu_france_std2a,taux_de_reussite_attendu_france_stmg,taux_de_reussite_attendu_france_sti,taux_de_reussite_attendu_france_stl,taux_de_reussite_attendu_france_st2s,taux_de_reussite_attendu_france_tmd,taux_de_reussite_attendu_france_sthr,taux_de_reussite_attendu_france_toutes_series,taux_de_mentions_l,taux_de_mentions_es,taux_de_mentions_s,taux_de_mentions_sti2d,taux_de_mentions_std2a,taux_de_mentions_stmg,taux_de_mentions_stl,taux_de_mentions_st2s,taux_de_mentions_tmd,taux_de_mentions_sthr,taux_de_mentions_toutes_series,taux_de_mentions_attendu_l,taux_de_mentions_attendu_es,taux_de_mentions_attendu_s,taux_de_mentions_attendu_sti2d,taux_de_mentions_attendu_std2a,taux_de_mentions_attendu_stmg,taux_de_mentions_attendu_stl,taux_de_mentions_attendu_st2s,taux_de_mentions_attendu_tmd,taux_de_mentions_attendu_sthr,taux_de_mentions_attendu_toutes_series,structure_pedagogique_5_groupes,structure_pedagogique_7_groupes,pourcentage_bacheliers_sortants_2de_1re_terminale_etablissement,pourcentage_bacheliers_sortants_terminales_etablissement,pourcentage_bacheliers_sortants_2de_1re_terminale_acad,pourcentage_bacheliers_sortants_terminale_acad,pourcentage_bacheliers_sortants_2de_1re_terminale_france,pourcentage_bacheliers_sortants_terminale_france,effectif_de_seconde,effectif_de_premiere,effectif_de_terminale,taux_d_acces_2nde_bac,taux_d_acces_attendu_acad_2nde_bac,taux_d_acces_attendu_france_2nde_bac,taux_d_acces_1ere_bac,taux_d_acces_attendu_acad_1ere_bac,taux_d_acces_attendu_france_1ere_bac,taux_d_acces_terminale_bac,taux_d_acces_attendu_france_terminale_bac,region,code_region,code_departement,libelle_departement,valeur_ajoutee_du_taux_de_reussite_toutes_series,valeur_ajoutee_du_taux_d_acces_2nde_bac,valeur_ajoutee_du_taux_de_mentions_toutes_series,valeur_ajoutee_du_taux_de_reussite_l,valeur_ajoutee_du_taux_de_reussite_es,valeur_ajoutee_du_taux_de_reussite_s,valeur_ajoutee_du_taux_de_reussite_stg,valeur_ajoutee_du_taux_de_reussite_sti2d,valeur_ajoutee_du_taux_de_reussite_std2a,valeur_ajoutee_du_taux_de_reussite_stmg,valeur_ajoutee_du_taux_de_reussite_sti,valeur_ajoutee_du_taux_de_reussite_stl,valeur_ajoutee_du_taux_de_reussite_st2s,valeur_ajoutee_du_taux_de_reussite_tmd,valeur_ajoutee_du_taux_de_reussite_sthr,valeur_ajoutee_du_taux_d_acces_1ere_bac,valeur_ajoutee_du_taux_d_acces_terminale_bac,valeur_ajoutee_du_taux_de_mentions_l,valeur_ajoutee_du_taux_de_mentions_es,valeur_ajoutee_du_taux_de_mentions_s,valeur_ajoutee_du_taux_de_mentions_sti2d,valeur_ajoutee_du_taux_de_mentions_std2a,valeur_ajoutee_du_taux_de_mentions_stmg,valeur_ajoutee_du_taux_de_mentions_stl,valeur_ajoutee_du_taux_de_mentions_st2s,valeur_ajoutee_du_taux_de_mentions_tmd,valeur_ajoutee_du_taux_de_mentions_sthr,presents_gnle,taux_de_reussite_gnle,valeur_ajoutee_du_taux_de_reussite_gnle,taux_de_mentions_gnle,v

In [45]:
df_ival_pro = pd.read_csv(
    './datasets/raw/fr-en-indicateurs-resultat-lycees-professionnels-2012-2022.csv',
    sep=';', low_memory=False)

snake_case(df_ival_pro)

df_ival_pro.head()

,etablissement,uai,annee,ville,code_commune,academie,departement,secteur,presents_specialites_pluri_technologiques_de_la_production,presents_transformations,presents_genie_civil_construction_bois,presents_materiaux_souples,presents_mecanique_electricite_electronique,presents_production,presents_specialites_plurivalentes_des_services,presents_echanges_et_gestion,presents_communication_et_information,presents_services_aux_personnes,presents_services_a_la_collectivite,presents_services,presents_toutes_series,taux_de_reussite_specialites_pluri_technologiques_de_la_production,taux_de_reussite_transformations,taux_de_reussite_genie_civil_construction_bois,taux_de_reussite_materiaux_souples,taux_de_reussite_mecanique_electricite_electronique,taux_de_reussite_production,taux_de_reussite_specialites_plurivalentes_des_services,taux_de_reussite_echanges_et_gestion,taux_de_reussite_communication_et_information,taux_de_reussite_services_aux_personnes,taux_de_reussite_services_a_la_collectivite,taux_de_reussite_services,taux_de_reussite_toutes_series,taux_de_reussite_attendu_france_specialites_pluri_technologiques_de_la_production,taux_de_reussite_attendu_france_transformations,taux_de_reussite_attendu_france_genie_civil_construction_bois,taux_de_reussite_attendu_france_materiaux_souples,taux_de_reussite_attendu_france_mecanique_electricite_electronique,taux_de_reussite_attendu_france_specialites_plurivalentes_des_services,taux_de_reussite_attendu_france_echanges_et_gestion,taux_de_reussite_attendu_france_communication_et_information,taux_de_reussite_attendu_france_services_aux_personnes,taux_de_reussite_attendu_france_services_a_la_collectivite,taux_de_reussite_attendu_acad_production,taux_de_reussite_attendu_acad_services,taux_de_reussite_attendu_acad_toutes_series,taux_de_reussite_attendu_france_production,taux_de_reussite_attendu_france_services,taux_de_reussite_attendu_france_toutes_series,pourcentage_bacheliers_sortants_2de_1re_terminale_etablissement,pourcentage_bacheliers_sortants_terminales_etablissement,pourcentage_bacheliers_sortants_2de_1re_terminale_acad,pourcentage_bacheliers_sortants_terminale_acad,pourcentage_bacheliers_sortants_2de_1re_terminale_france,pourcentage_bacheliers_sortants_terminale_france,effectif_de_seconde,effectif_de_premiere,effectif_de_terminale,taux_d_acces_2nde_bac,taux_d_acces_attendu_acad_2nde_bac,taux_d_acces_attendu_france_2nde_bac,taux_d_acces_1ere_bac,taux_d_acces_attendu_acad_1ere_bac,taux_d_acces_attendu_france_1ere_bac,taux_d_acces_terminale_bac,taux_d_acces_attendu_france_terminale_bac,taux_de_mentions_specialites_pluri_technologiques_de_la_production,taux_de_mentions_transformations,taux_de_mentions_genie_civil_construction_bois,taux_de_mentions_materiaux_souples,taux_de_mentions_mecanique_electricite_electronique,taux_de_mentions_specialites_plurivalentes_des_services,taux_de_mentions_echanges_et_gestion,taux_de_mentions_communication_et_information,taux_de_mentions_services_aux_personnes,taux_de_mentions_services_a_la_collectivite,taux_de_mentions_production,taux_de_mentions_services,taux_de_mentions_toutes_series,taux_de_mentions_attendu_specialites_pluri_technologiques_de_la_production,taux_de_mentions_attendu_transformations,taux_de_mentions_attendu_genie_civil_construction_bois,taux_de_mentions_attendu_materiaux_souples,taux_de_mentions_attendu_mecanique_electricite_electronique,taux_de_mentions_attendu_specialites_plurivalentes_des_services,taux_de_mentions_attendu_echanges_et_gestion,taux_de_mentions_attendu_communication_et_information,taux_de_mentions_attendu_services_aux_personnes,taux_de_mentions_attendu_services_a_la_collectivite,taux_de_mentions_attendu_production,taux_de_mentions_attendu_services,taux_de_mentions_attendu_toutes_series,structure_pedagogique_7_groupes,region,code_region,libelle_departement,code_departement,valeur_ajoutee_du_taux_de_reussite_specialites_pluri_technologiques_de_la_production,valeur_ajoutee_du_taux_de_reussite_transformations,valeur_ajoutee_du_taux_de

Les fichiers ne seront pas concaténés, cette fois-ci, car les indicateurs sont relatifs aux séries et ces dernières sont différentes selon le type d'enseignement.

### Dimensions

In [46]:
print(
    f'Dimensions des DataFrames :\n'
    f'- df_ival_gt : {df_ival_gt.shape}\n'
    f'- df_ival_pro : {df_ival_pro.shape}'
    )

Dimensions des DataFrames :
- df_ival_gt : (25477, 145)
- df_ival_pro : (22227, 130)


### Tri des variables

Encore une fois nous n'avons pas besoin des variables relatives aux informations géographiques car nous les avons dans un autre fichier.<br>
Avant de les enlever, on supprime les enregistrements des DROM-COM.

In [47]:
df_ival_gt['code_departement'].unique()

array(['41', '42', '43', '44', '46', '48', '49', '50', '52', '53', '54',
       '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '67', '68', '69', '71', '72', '73', '74', '75', '76', '77',
       '78', '79', '80', '82', '83', '84', '85', '86', '87', '88', '89',
       '90', '91', '92', '93', '94', '95', '2B', '971', '972', '974',
       '01', '02', '03', '04', '06', '07', '08', '09', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35',
       '36', '37', '38', '39', '40', '45', '47', '51', '05', '2A', '973',
       '81', '976', '70'], dtype=object)

In [48]:
# Crée la liste des départements à supprimer
index_names = df_ival_gt.loc[
    (df_ival_gt['code_departement'].str.startswith('9')) 
     & (df_ival_gt['code_departement'].str.len() == 3)
].index

# Supprime les enregistrements
df_ival_gt = df_ival_gt.drop(index=index_names).reset_index(drop=True)
del index_names

In [49]:
df_ival_pro['code_departement'].unique()

array(['94', '95', '2A', '2B', '971', '972', '973', '974', '976', '02',
       '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13',
       '14', '16', '17', '18', '19', '21', '22', '23', '24', '25', '26',
       '27', '28', '29', '30', '31', '33', '34', '35', '37', '38', '39',
       '40', '42', '43', '44', '45', '46', '48', '49', '50', '51', '52',
       '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63',
       '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74',
       '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85',
       '86', '87', '88', '89', '91', '92', '93', '01', '15', '32', '41',
       '36', '47', '90'], dtype=object)

In [50]:
# Crée la liste des départements à supprimer
index_names = df_ival_pro.loc[
    (df_ival_pro['code_departement'].str.startswith('9')) 
     & (df_ival_pro['code_departement'].str.len() == 3)
].index

# Supprime les enregistrements
df_ival_pro = df_ival_pro.drop(index=index_names).reset_index(drop=True)
del index_names

In [51]:
df_ival_gt = df_ival_gt.drop(
    columns={
        'etablissement', 'ville', 'code_commune', 'academie',
        'departement', 'region', 'code_region', 'code_departement',
        'libelle_departement'
    }
)

df_ival_pro = df_ival_pro.drop(
    columns={
        'etablissement', 'ville', 'code_commune', 'academie',
        'departement', 'region', 'code_region', 'code_departement',
        'libelle_departement'
    }
)

Puis on ne garde que les variables qui reflètent les résultats pour l'ensemble des séries. En ce qui concerne les résultats attendus, on ne conserve que le regroupement France.

In [52]:
df_ival_gt = df_ival_gt[
    [
        'annee',
        'uai',
        'secteur',
        'presents_toutes_series',
        'taux_de_reussite_toutes_series',
        'taux_de_reussite_attendu_france_toutes_series',
        'taux_de_mentions_toutes_series',
        'taux_de_mentions_attendu_toutes_series',
        'effectif_de_seconde',
        'effectif_de_premiere',
        'effectif_de_terminale',
        'taux_d_acces_2nde_bac',
        'taux_d_acces_attendu_france_2nde_bac',
        'valeur_ajoutee_du_taux_de_reussite_toutes_series',
        'valeur_ajoutee_du_taux_d_acces_2nde_bac',
        'valeur_ajoutee_du_taux_de_mentions_toutes_series'
    ]
]

df_ival_pro = df_ival_pro[
    [
        'annee',
        'uai',
        'secteur',
        'presents_toutes_series',
        'taux_de_reussite_toutes_series',
        'taux_de_reussite_attendu_france_toutes_series',
        'taux_de_mentions_toutes_series',
        'taux_de_mentions_attendu_toutes_series',
        'effectif_de_seconde',
        'effectif_de_premiere',
        'effectif_de_terminale',
        'taux_d_acces_2nde_bac',
        'taux_d_acces_attendu_france_2nde_bac',
        'valeur_ajoutee_du_taux_de_reussite_toutes_series',
        'valeur_ajoutee_du_taux_d_acces_2nde_bac',
        'valeur_ajoutee_du_taux_de_mentions_toutes_series'
    ]
]

On supprime aussi les enregistrements antérieurs à 2017.

In [53]:
# Crée la liste des années à supprimer
index_names_gt = df_ival_gt.loc[df_ival_gt['annee'] <= 2017].index
index_names_pro = df_ival_pro.loc[df_ival_pro['annee'] <= 2017].index

# Supprime les enregistrements
df_ival_gt = df_ival_gt.drop(index=index_names_gt).reset_index(drop=True)
df_ival_pro = df_ival_pro.drop(index=index_names_pro).reset_index(drop=True)
del index_names_gt, index_names_pro

On retouche les valeurs de « secteur ».

In [54]:
df_ival_gt['secteur'] = df_ival_gt['secteur'].map(
    {'public': 'Public', 'privé sous contrat': 'Privé'}
)
df_ival_pro['secteur'] = df_ival_pro['secteur'].map(
    {'public': 'Public', 'privé sous contrat': 'Privé'}
)

### Valeurs manquantes

In [55]:
df_ival_gt.isna().sum()

annee                                                  0
uai                                                    0
secteur                                                0
presents_toutes_series                                 0
taux_de_reussite_toutes_series                         0
taux_de_reussite_attendu_france_toutes_series       6676
taux_de_mentions_toutes_series                         0
taux_de_mentions_attendu_toutes_series              6676
effectif_de_seconde                                    3
effectif_de_premiere                                   3
effectif_de_terminale                                  3
taux_d_acces_2nde_bac                                 34
taux_d_acces_attendu_france_2nde_bac                6709
valeur_ajoutee_du_taux_de_reussite_toutes_series       3
valeur_ajoutee_du_taux_d_acces_2nde_bac               38
valeur_ajoutee_du_taux_de_mentions_toutes_series       3
dtype: int64

In [56]:
df_ival_pro.isna().sum()

annee                                                  0
uai                                                    0
secteur                                                0
presents_toutes_series                                 0
taux_de_reussite_toutes_series                         0
taux_de_reussite_attendu_france_toutes_series       5755
taux_de_mentions_toutes_series                         0
taux_de_mentions_attendu_toutes_series              5755
effectif_de_seconde                                    3
effectif_de_premiere                                   3
effectif_de_terminale                                  3
taux_d_acces_2nde_bac                                  0
taux_d_acces_attendu_france_2nde_bac                5755
valeur_ajoutee_du_taux_de_reussite_toutes_series       4
valeur_ajoutee_du_taux_d_acces_2nde_bac                2
valeur_ajoutee_du_taux_de_mentions_toutes_series       4
dtype: int64

Les valeurs sont manquantes dans les taux attendus. Or, les valeurs sont liées par la relation suivante : $valeur\ ajoutée = taux - taux\ attendu$.<br>
On peut donc se passer des taux attendus.

In [57]:
# Crée la liste des variables à supprimer
variables_list = [
    'taux_de_reussite_attendu_france_toutes_series',
    'taux_de_mentions_attendu_toutes_series',
    'taux_d_acces_attendu_france_2nde_bac'
]

# Supprime les variables
df_ival_gt = df_ival_gt.drop(columns=variables_list)
df_ival_pro = df_ival_pro.drop(columns=variables_list)

del variables_list

### Doublons

In [58]:
df_ival_gt.duplicated().sum()

0

In [59]:
df_ival_pro.duplicated().sum()

0

Il n'y a pas de doublons.

### Type des données

In [60]:
df_ival_gt.dtypes

annee                                                 int64
uai                                                  object
secteur                                              object
presents_toutes_series                                int64
taux_de_reussite_toutes_series                        int64
taux_de_mentions_toutes_series                      float64
effectif_de_seconde                                 float64
effectif_de_premiere                                float64
effectif_de_terminale                               float64
taux_d_acces_2nde_bac                                object
valeur_ajoutee_du_taux_de_reussite_toutes_series     object
valeur_ajoutee_du_taux_d_acces_2nde_bac              object
valeur_ajoutee_du_taux_de_mentions_toutes_series     object
dtype: object

In [61]:
df_ival_pro.dtypes

annee                                                 int64
uai                                                  object
secteur                                              object
presents_toutes_series                                int64
taux_de_reussite_toutes_series                        int64
taux_de_mentions_toutes_series                      float64
effectif_de_seconde                                 float64
effectif_de_premiere                                float64
effectif_de_terminale                               float64
taux_d_acces_2nde_bac                               float64
valeur_ajoutee_du_taux_de_reussite_toutes_series     object
valeur_ajoutee_du_taux_d_acces_2nde_bac              object
valeur_ajoutee_du_taux_de_mentions_toutes_series     object
dtype: object

On apporte quelques modifications de type.

In [62]:
df_ival_gt = df_ival_gt.astype(
    {'annee': 'category', 'secteur': 'category'}
)

df_ival_pro = df_ival_pro.astype(
    {'annee': 'category', 'secteur': 'category'}
)

In [63]:
df_ival_gt = df_ival_gt.astype(
    {'presents_toutes_series': 'int16',
     'effectif_de_seconde': pd.Int16Dtype(),
     'effectif_de_premiere': pd.Int16Dtype(),
     'effectif_de_terminale': pd.Int16Dtype()
    }
)

df_ival_pro = df_ival_pro.astype(
    {'presents_toutes_series': 'int16',
     'effectif_de_seconde': pd.Int16Dtype(),
     'effectif_de_premiere': pd.Int16Dtype(),
     'effectif_de_terminale': pd.Int16Dtype()
    }
)

In [64]:
try:
    df_ival_gt = df_ival_gt.astype(
        {'taux_de_reussite_toutes_series': 'float',
        'valeur_ajoutee_du_taux_de_reussite_toutes_series': 'float',
        'valeur_ajoutee_du_taux_d_acces_2nde_bac': 'float',
        'valeur_ajoutee_du_taux_de_mentions_toutes_series': 'float',
        'taux_d_acces_2nde_bac': 'float'
        }
    )

    df_ival_pro = df_ival_pro.astype(
        {'taux_de_reussite_toutes_series': 'float',
        'valeur_ajoutee_du_taux_de_reussite_toutes_series': 'float',
        'valeur_ajoutee_du_taux_d_acces_2nde_bac': 'float',
        'valeur_ajoutee_du_taux_de_mentions_toutes_series': 'float',
        'taux_d_acces_2nde_bac': 'float'
        }
    )
except ValueError as erreur:
    print(erreur)

could not convert string to float: 'ND'


La conversion ne peut pas avoir lieu car certaines fois les variables ont la valeur « ND ».

In [65]:
df_ival_gt = df_ival_gt.replace('ND', np.NaN)
df_ival_pro = df_ival_pro.replace('ND', np.NaN)

try:
    df_ival_gt = df_ival_gt.astype(
        {'taux_de_reussite_toutes_series': 'float',
        'valeur_ajoutee_du_taux_de_reussite_toutes_series': 'float',
        'valeur_ajoutee_du_taux_d_acces_2nde_bac': 'float',
        'valeur_ajoutee_du_taux_de_mentions_toutes_series': 'float',
        'taux_d_acces_2nde_bac': 'float'
        }
    )

    df_ival_pro = df_ival_pro.astype(
        {'taux_de_reussite_toutes_series': 'float',
        'valeur_ajoutee_du_taux_de_reussite_toutes_series': 'float',
        'valeur_ajoutee_du_taux_d_acces_2nde_bac': 'float',
        'valeur_ajoutee_du_taux_de_mentions_toutes_series': 'float',
        'taux_d_acces_2nde_bac': 'float'
        }
    )
except ValueError as erreur:
    print(erreur)

could not convert string to float: '.'


La conversion a encore échoué car il y la valeur « . ».

In [66]:
df_ival_gt = df_ival_gt.replace('.', np.NaN)
df_ival_pro = df_ival_pro.replace('.', np.NaN)

try:
    df_ival_gt = df_ival_gt.astype(
        {'taux_de_reussite_toutes_series': 'float',
        'valeur_ajoutee_du_taux_de_reussite_toutes_series': 'float',
        'valeur_ajoutee_du_taux_d_acces_2nde_bac': 'float',
        'valeur_ajoutee_du_taux_de_mentions_toutes_series': 'float',
        'taux_d_acces_2nde_bac': 'float'
        }
    )

    df_ival_pro = df_ival_pro.astype(
        {'taux_de_reussite_toutes_series': 'float',
        'valeur_ajoutee_du_taux_de_reussite_toutes_series': 'float',
        'valeur_ajoutee_du_taux_d_acces_2nde_bac': 'float',
        'valeur_ajoutee_du_taux_de_mentions_toutes_series': 'float',
        'taux_d_acces_2nde_bac': 'float'
        }
    )
except ValueError as erreur:
    print(erreur)

La dernière étape consiste à diviser les taux par 100 pour qu'ils puissent être interprétés comme des pourcentages par la suite.

In [67]:
df_ival_gt.dtypes

annee                                               category
uai                                                   object
secteur                                             category
presents_toutes_series                                 int16
taux_de_reussite_toutes_series                       float64
taux_de_mentions_toutes_series                       float64
effectif_de_seconde                                    Int16
effectif_de_premiere                                   Int16
effectif_de_terminale                                  Int16
taux_d_acces_2nde_bac                                float64
valeur_ajoutee_du_taux_de_reussite_toutes_series     float64
valeur_ajoutee_du_taux_d_acces_2nde_bac              float64
valeur_ajoutee_du_taux_de_mentions_toutes_series     float64
dtype: object

In [68]:
# Crée la liste des variables à diviser par 100
variables_list = [
    'taux_de_reussite_toutes_series',
    'taux_de_mentions_toutes_series',
    'taux_d_acces_2nde_bac',
    'valeur_ajoutee_du_taux_de_reussite_toutes_series',
    'valeur_ajoutee_du_taux_d_acces_2nde_bac',
    'valeur_ajoutee_du_taux_de_mentions_toutes_series'
]

# Divise les valeurs par 100
df_ival_gt = df_ival_gt.apply(
    lambda x: (x / 100) if x.name in variables_list else x)

df_ival_pro = df_ival_pro.apply(
    lambda x: (x / 100) if x.name in variables_list else x)

del variables_list

### Exportation CSV

In [69]:
df_ival_gt.to_csv('./datasets/ival-lycees-gt-2017-2022-hors-drom-com.csv',
                    index=False, decimal=',', quoting=1)
df_ival_pro.to_csv('./datasets/ival-lycees-pro-2017-2022-hors-drom-com.csv',
                    index=False, decimal=',', quoting=1)